In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url = 'https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1/plus/?saison_id=2018'

r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [3]:
vereine = soup.find_all('td', {'class':'hauptlink no-border-links show-for-small show-for-pad'})

for team in vereine:
    print(team.text)

BSC Young Boys
FC Basel
FC Sion
Grasshoppers
FC Luzern
FC Zürich
FC St. Gallen
FC Lugano
FC Thun
NE Xamax


In [4]:
marktwerte = soup.find_all('td', {'class':'rechts show-for-small show-for-pad nowrap'})

for wert in marktwerte[2:][::2]:
    print(float(wert.text.replace(' Mio. €', '').replace(',', '.')))

69.25
51.0
28.05
26.6
22.95
21.6
18.9
18.3
18.25
10.15


In [5]:
teamwertliste = []
for team, wert in zip(vereine, marktwerte[2:][::2]):
    team = team.text
    wert = float(wert.text.replace(' Mio. €', '').replace(',', '.'))
    dictionary = {'Mannschaft': team,
                 'Marktwert': wert}
    
    teamwertliste.append(dictionary)

df_marktwerte = pd.DataFrame(teamwertliste)
df_marktwerte

,Mannschaft,Marktwert
0,BSC Young Boys,69.25
1,FC Basel,51.00
2,FC Sion,28.05
3,Grasshoppers,26.60
4,FC Luzern,22.95
5,FC Zürich,21.60
6,FC St. Gallen,18.90
7,FC Lugano,18.30
8,FC Thun,18.25
9,NE Xamax,10.15


In [6]:
vereine2 = soup.find_all('td', {'class':'no-border-links hauptlink'})

for team2 in vereine2:
    print(team2.text)


BSC Young Boys  

FC Basel 

FC Thun 

FC Zürich  

FC Luzern 

FC St. Gallen 

FC Sion 

FC Lugano 

Grasshoppers 

NE Xamax   


In [7]:
alle_tabellen = soup.find_all('div', {'class':'responsive-table'})
tabelle2 = alle_tabellen[1]
reihen = tabelle2.find_all('tr')

for reihe in reihen[1:]:
    alle_tds = reihe.find_all('td')
    punkte = alle_tds[-1]
    print(punkte.text)

49
30
28
25
25
23
21
19
17
13


In [8]:
teampunkte = {}

for reihe in reihen[1:]:
    alle_tds = reihe.find_all('td')
    punkte = alle_tds[-1]
    team = alle_tds[2]
    teampunkte[team.text.strip()] = int(punkte.text)

print(teampunkte)

{'BSC Young Boys': 49, 'FC Basel': 30, 'FC Thun': 28, 'FC Zürich': 25, 'FC Luzern': 25, 'FC St. Gallen': 23, 'FC Sion': 21, 'FC Lugano': 19, 'Grasshoppers': 17, 'NE Xamax': 13}


In [10]:
teamwertliste = []
for team, wert in zip(vereine, marktwerte[2:][::2]):
    team = team.text
    wert = float(wert.text.replace(' Mio. €', '').replace(',', '.'))
    row = teampunkte[team]
    dictionary = {'Club': team, 'Marktwert': wert, 'Punkte': row}
    
    teamwertliste.append(dictionary)
    
pd.DataFrame(teamwertliste)

,Club,Marktwert,Punkte
0,BSC Young Boys,69.25,49
1,FC Basel,51.00,30
2,FC Sion,28.05,21
3,Grasshoppers,26.60,17
4,FC Luzern,22.95,25
5,FC Zürich,21.60,25
6,FC St. Gallen,18.90,23
7,FC Lugano,18.30,19
8,FC Thun,18.25,28
9,NE Xamax,10.15,13


In [11]:
df_18_19 = pd.DataFrame(teamwertliste)

df_18_19['Geld_pro_Punkt'] =  df_18_19['Marktwert'] / df_18_19['Punkte']
df_18_19

,Club,Marktwert,Punkte,Geld_pro_Punkt
0,BSC Young Boys,69.25,49,1.413265
1,FC Basel,51.00,30,1.700000
2,FC Sion,28.05,21,1.335714
3,Grasshoppers,26.60,17,1.564706
4,FC Luzern,22.95,25,0.918000
5,FC Zürich,21.60,25,0.864000
6,FC St. Gallen,18.90,23,0.821739
7,FC Lugano,18.30,19,0.963158
8,FC Thun,18.25,28,0.651786
9,NE Xamax,10.15,13,0.780769


In [12]:
df_18_19.sort_values('Geld_pro_Punkt', ascending=False)

,Club,Marktwert,Punkte,Geld_pro_Punkt
1,FC Basel,51.00,30,1.700000
3,Grasshoppers,26.60,17,1.564706
0,BSC Young Boys,69.25,49,1.413265
2,FC Sion,28.05,21,1.335714
7,FC Lugano,18.30,19,0.963158
4,FC Luzern,22.95,25,0.918000
5,FC Zürich,21.60,25,0.864000
6,FC St. Gallen,18.90,23,0.821739
9,NE Xamax,10.15,13,0.780769
8,FC Thun,18.25,28,0.651786


In [13]:
df_18_19_winterpause = df_18_19.sort_values('Geld_pro_Punkt', ascending=False)

df_18_19_winterpause.to_csv('winterpause_1819.csv')